In [ ]:
using DynamicPolynomials
using SumOfSquares
using CSDP
using JuMP
using Plots
using LinearAlgebra
using DifferentialEquations
using PyCall

$$x' = u (1 - \theta^2/2), y' = u (\theta - \theta^3/3), \theta' = u_2 $$ 

# 0. Init system variable and parameters

In [ ]:
#init state and input variables
@polyvar x[1:3]
@polyvar u[1:2]

#define system dynamics
vectorField = [ (1 - x[3]^2/2), (x[3] - x[3]^3/3), u[1] ]

#define unsafe set (obstacle)
g = 0.1^2 - x[1]^2 - x[2]^2

#state and input bounds
bounds = [[-10., 10.], [-10., 10.], [-π, π] ]
u_bounds = [[-0.1,0.1],[-0.1,0.1]]

In [ ]:
function prepare_domain(x::Vector{<:Variable}, bounds::Vector{Vector{Float64}})
   poly_list = [ 
        @set(x[i] - l >= 0) ∩ @set(u - x[i] >= 0 ) ∩ @set((x[i] -l)*(u-x[i]) >= 0)
        for (i, (l, u)) in enumerate(bounds)
        ] 
   poly_list
end

In [ ]:
#instantiate parameters
λ = 1
ϵ = 1
max_degree=4
U = [[-0.1, -0.1], [0.1, 0.1]]

In [ ]:
# generate test points
function get_random(limits::Vector{Vector{Float64}}, g::Polynomial)
    function get_random_scalar(lb, ub )
        lb + rand()*(ub - lb) 
    end
    while (true)
        pt = [get_random_scalar(l[1], l[2]) for l in limits]
        if g(pt[1], pt[2]) >= 0
            continue
        else
            return pt
        end
    end
end

test_pts = [ get_random(bounds, g) for _ in 1:25];

# 1. Computing initial set of barriers for each input U_i

In [ ]:
# function to generate initial barriers for each input

function generate_barrier(x, u, bounds, g, vectorField, U, test_pts; max_degree=4,ϵ = 0.25, λ = 0.1, γ = 10.)
    solver = optimizer_with_attributes(CSDP.Optimizer)
    model = SOSModel(solver)
    dom_list = prepare_domain(x, bounds)
    dom = reduce( (s1, s2) -> s1 ∩ s2, dom_list)
    #println("Domain: $dom")
    # negative inside the obstacle
    monos = monomials(x, 0:max_degree)
    N = length(monos) 
    @variable(model, -γ <= c[1:N] <= γ)
    
    B = polynomial(c[1:end], monos) 
    #negative inside the domain
    @constraint(model, cons1, B <= -ϵ, domain=dom ∩ @set(g >= 0) )
    B_dot = dot(differentiate(B,x), vectorField)
    B_dot_with_u = subs(B_dot, u => U)
    @constraint(model, cons2, B_dot_with_u >= λ * B, domain=dom)
    
    set_objective_sense(model, MOI.FEASIBILITY_SENSE)
    objective_fn = sum([B(pt...) for pt in test_pts])
    @objective(model, Max, objective_fn) # keep as many points outside the barrier as you can
    JuMP.optimize!(model)
    stat = JuMP.primal_status(model)
    if stat != FEASIBLE_POINT
        return missing
    end
    # found feasible point
    println(solution_summary(model))
    lm = [lagrangian_multipliers(cons1)]
    push!(lm, lagrangian_multipliers(cons2))
    value(B), value(B_dot), lm
end

In [ ]:
# function to comupte transit time for each barrier 

function refine_barrier_dn(x, u, bounds, u_bounds, g, vectorField, B, B_dot; ϵ = 0.25, κ = .1)
    solver = optimizer_with_attributes(CSDP.Optimizer)
    model = SOSModel(solver)
    dom_list = prepare_domain(x, bounds)
    dom_list_u = prepare_domain(u, u_bounds)
    dom_list = append!(dom_list, dom_list_u)
    dom = reduce( (s1, s2) -> s1 ∩ s2, dom_list)
    #println("Domain: $dom")
    # negative inside the obstacle
    monos = monomials(x, 0:max_degree)
    N = length(monos) 
    @variable(model, η)
    @variable(model, δ >=0)
    dom3 = dom ∩ @set(B >= 0) ∩ @set(B <= κ)
    @constraint(model, -η*B - δ <= B_dot, domain=dom3)
    @constraint(model, η*κ + δ >= 0)
    @objective(model, Min, δ)
    JuMP.optimize!(model)
    stat = JuMP.primal_status(model)
    if stat != FEASIBLE_POINT
        return missing
    end
    # found feasible point
    println(solution_summary(model))
    println("η = $(JuMP.value(η))")
    println("δ = $(JuMP.value(δ))")
    τ =  (κ)/max(JuMP.value(δ),JuMP.value(η) *  κ + JuMP.value(δ))
    println("τd = $(JuMP.value(τ))")
    return τ
end

In [ ]:
elapsed1 = @elapsed begin
B_0, B_0d, lm0 = generate_barrier(x, u, bounds, g, vectorField, U[1], test_pts; max_degree = 2)
end
display(B_0)
display(B_0d)
t_0d = refine_barrier_dn(x, u, bounds, u_bounds, g, vectorField, B_0,  B_0d)

if (!ismissing(B_0))
    test_pts = filter!(pt -> B_0(pt...) <= 0., test_pts)
end

In [ ]:
elapsed2 = @elapsed begin
B_1, B_1d, lm1 = generate_barrier(x, u, bounds, g, vectorField, U[2], test_pts; max_degree=2)
end
display(B_1)
display(B_1d)
t_1d = refine_barrier_dn(x, u, bounds, u_bounds, g, vectorField, B_1,  B_1d)

if (!ismissing(B_1))
    test_pts = filter!(pt -> B_1(pt...) <= 0., test_pts)
end

# 2. Computing Successive Barriers

In [ ]:
# functions to compute successive barriers

In [ ]:
function generate_successive_barrier(x::Vector{<:Variable}, u::Vector{<:Variable}, 
        bounds::Vector{Vector{Float64}}, g::Polynomial, 
        vectorField::Vector{<:Polynomial}, U::Vector{Float64}, 
        test_pts::Vector{Vector{Float64}}, ancestors::Vector{<:Polynomial}, 
        pt_to_eliminate::Vector{Float64}; 
        max_degree=4,ϵ = 1, λ = 1, γ = 10., κ = .1)
    solver = optimizer_with_attributes(CSDP.Optimizer, MOI.Silent() => true)
    model = SOSModel(solver)
    dom_list = prepare_domain(x, bounds)
    dom = reduce( (s1, s2) -> s1 ∩ s2, dom_list)
    #println("Domain: $dom")

    monos = monomials(x, 0:max_degree)
    N = length(monos) 
    @variable(model, -γ <= c[1:N] <= γ)
    
    B = polynomial(c[1:end], monos) 
    # negative inside the obstacle
    @constraint(model, cons1, B <= -ϵ, domain=dom ∩ @set(g >= 0) )
    
    # dynamics constraints
    B_dot = dot(differentiate(B,x), vectorField)
    B_dot_with_u = subs(B_dot, u => U)
    if size(ancestors)[1] >= 1
        new_domain = dom ∩ (reduce(∩, [@set(B <= 0) for B in ancestors]))
    else
        new_domain = dom 
    end
    @constraint(model, cons2, B_dot_with_u >= λ * B, domain=new_domain)
    
    # eliminate the point we would like to eliminate
    @constraint(model, B(pt_to_eliminate...) >= ϵ)
    set_objective_sense(model, MOI.FEASIBILITY_SENSE)
    
    # maximize the sum of values for all test points
    objective_fn = sum([B(pt...) for pt in test_pts])
    @objective(model, Max, objective_fn) # keep as many points outside the barrier as you can
    JuMP.optimize!(model)
    stat = JuMP.primal_status(model)
    if stat != FEASIBLE_POINT
        return missing
    end
    # found feasible point
    println(solution_summary(model))
    lm = [lagrangian_multipliers(cons1)]
    push!(lm, lagrangian_multipliers(cons2))
    value(B), value(B_dot), lm
end

In [ ]:
function refine_barrier_succ_dn(x, u, bounds, u_bounds, g, vectorField, B, B_dot, ancestors; ϵ = 0.25, κ = .1)
    solver = optimizer_with_attributes(CSDP.Optimizer)
    model = SOSModel(solver)
    dom_list = prepare_domain(x, bounds)
    dom_list_u = prepare_domain(u, u_bounds)
    dom_list = append!(dom_list, dom_list_u)
    dom = reduce( (s1, s2) -> s1 ∩ s2, dom_list)
    #println("Domain: $dom")
    # negative inside the obstacle
    monos = monomials(x, 0:max_degree)
    N = length(monos) 
    @variable(model, η)
    @variable(model, δ>=0)

    if size(ancestors)[1] >= 1
        new_domain = dom ∩ (reduce(∩, [@set(b <= 0) for b in ancestors]))
    else
        new_domain = dom 
    end
    
    dom3 = new_domain ∩ @set(B >= 0) ∩ @set(B <= κ)
    @constraint(model, -η*B - δ <= B_dot, domain=dom3)
    @constraint(model, η*κ + δ >= 0)

    #set_objective_sense(model, MOI.FEASIBILITY_SENSE)
    @objective(model, Min, δ)
    JuMP.optimize!(model)
    stat = JuMP.primal_status(model)
    if stat != FEASIBLE_POINT
        return missing
    end
    # found feasible point
    println(solution_summary(model))
    println("η = $(value(η))")
    println("δ = $(value(δ))")
    τ =  (κ)/max(value(δ),value(η) *  κ + value(δ))
    println("τd = $(value(τ))")
    return τ
end

In [ ]:
function compute_next_level_barriers(x::Vector{<:Variable}, u::Vector{<:Variable}, bounds::Vector{Vector{Float64}},
                                    u_bounds::Vector{Vector{Float64}},
                                    g::Polynomial, vectorField::Vector{<:Polynomial}, 
                                    U::Vector{Vector{Float64}}, test_pts::Vector{Vector{Float64}}, 
                                    ancestors::Vector{<:Polynomial}) 
    eliminated = []
    second_level_barriers=[]
    for (j,pt) in enumerate(test_pts) 
        if j in eliminated
            # ignore the ones already eliminated
            continue
        end
        # go through each control 
        for u_val in U
            # generate a barrier 
            B = generate_successive_barrier(x, u, bounds, g, vectorField, u_val, test_pts, ancestors, pt)
            # check if we found something
            if !ismissing(B)
                println("Bingo: Found $B")
                useful = false # check if it is actually useful
                for (k,pt_new) in enumerate(test_pts)
                    if k in eliminated
                        continue
                    end
                   if (B[1](pt_new...) >= 0.)
                    push!(eliminated, k)   
                    println("\t eliminated $pt_new")
                    useful=true
                   end
                end
                if (useful)
                    println("Num remaining = $(size(test_pts)[1] - size(eliminated)[1])")
                    td = refine_barrier_succ_dn(x, u, bounds, u_bounds, g, vectorField, B[1],  B[2], ancestors)         
                    push!(second_level_barriers, (B[1], u_val, td,  B[3]))
                    break
                end
            end
        end
    end
    new_test_pts = [pt for (j, pt) in enumerate(test_pts) if !(j in eliminated)]
    return (second_level_barriers, new_test_pts)
end

In [ ]:
zero_level_barriers::Vector{Polynomial} =[B_0, B_1]
print(size(zero_level_barriers))
s_elapsed = @elapsed begin
(first_level_barriers, test_pts_1) = compute_next_level_barriers(x, u, bounds, u_bounds, g, vectorField, U, test_pts, zero_level_barriers)
end

In [ ]:
test_pts_1 = [ get_random(bounds, g) for _ in 1:25];

In [ ]:
all_barriers1::Vector{Polynomial} = []
append!(all_barriers1, zero_level_barriers)
append!(all_barriers1, [B for (B, _) in first_level_barriers])
(second_level_barriers, test_pts_2) = compute_next_level_barriers(x, u, bounds, u_bounds, g, vectorField, U, test_pts_1, all_barriers1)

# 3. Plots

In [ ]:
# plotting functions

function circleShape(x,y, r)
    θ = LinRange(0, 2*π, 500)
    x .+ r*cos.(θ), y .+ r*sin.(θ)
end

function plot_ci_region3lev(limits::Tuple{Float64,Float64}, lev1_barriers::Vector{<:Polynomial}, lev2_barriers::Vector{<:Polynomial}, lev3_barriers::Vector{<:Polynomial}; δ = 0.1, theta_val=0.0, filestem="ics")
    plot(xlims=limits, ylims=limits)
    rectangle(w, h, x, y) = Shape(x .+ [0,w,w,0], y .+ [0,0,h,h])
    for x in limits[1]:δ:limits[2]
        for y in limits[1]:δ:limits[2]
            if (any([ B(x, y, theta_val) > 0. for B in lev1_barriers]))
                plot!(rectangle(δ, δ, x-δ, y-δ), label=false, fill=:seagreen1, opacity=0.5,linecolor=:seagreen1)
            else
                if (any([ B(x, y, theta_val) > 0. for B in lev2_barriers]))
                    plot!(rectangle(δ, δ, x-δ, y-δ), label=false, fill=:limegreen, linecolor=:limegreen)
                else
                    if (any([ B(x, y, theta_val) > 0. for B in lev3_barriers]))
                        plot!(rectangle(δ, δ, x-δ, y-δ), label=false, fill=:darkgreen, linecolor=:darkgreen)
                    end
                end 
            end
        end
    end
    plot!(circleShape(0,0,0.2), seriestype =[:shape], lw=0.5, c=:black, linecolor=:black, legend=false, aspectratio=1  )
    plot!(xlims=limits, ylims=limits)
    filename="figures/$filestem-theta-$(round(theta_val; digits=2)).png"
    savefig(filename)
    plot!(xlims=limits, ylims=limits)
end

In [ ]:
lev2_barriers::Vector{Polynomial} = [B for (B,_) in second_level_barriers]
lev1_barriers::Vector{Polynomial} = [B for (B,_) in first_level_barriers]
plot_ci_region3lev((-10., 10.0), [B_0, B_1], lev1_barriers, lev2_barriers; theta_val=3.14, δ=0.25, filestem="Figure3a-running-ex-lev2")

In [ ]:
plot_ci_region3lev((-10., 10.0), [B_0, B_1], lev1_barriers, lev2_barriers; theta_val=1.57, δ=0.25, filestem="Figure3b-running-ex-lev2")

In [ ]:
plot_ci_region3lev((-10., 10.0), [B_0, B_1], lev1_barriers, lev2_barriers; theta_val=0.0, δ=0.25, filestem="Figure3c-running-ex-lev2")

In [ ]:
plot_ci_region3lev((-10., 10.0), [B_0, B_1], lev1_barriers, lev2_barriers; theta_val=-1.57, δ=0.25, filestem="Figure3d-running-ex-lev2")

In [ ]:
plot_ci_region3lev((-10., 10.0), [B_0, B_1], lev1_barriers, lev2_barriers; theta_val=-3.14, δ=0.25, filestem="Figure3e-running-ex-lev2")

In [ ]:
# minimum transit time over all barriers
t0_min = minimum([t_0d, t_1d]);
t1=[]
for i in first_level_barriers
    if (!ismissing(i[3]))
        t1 = append!(t1,i[3])
    end
end
t1 = filter(x -> x > 0, t1)
t1_min = minimum(t1);
t2=[]
for i in second_level_barriers
    if (!ismissing(i[3]))
        t2 = append!(t2,i[3])
    end
end

println("τ_min = $(minimum([t0_min, t1_min]))")

# 4. Benchmarks

In [ ]:
# generating data for Table 1 in the paper (row for current system)

In [ ]:
println("Time taken for B1: $(elapsed1+elapsed2)")
println("# barriers B1: $(size(zero_level_barriers))")
println("Time taken for B2: $(s_elapsed)")
println("# barriers B2: $(size(first_level_barriers))")

# 5. Verification

In [ ]:
"""
Certifying barriers using constraint PSD check
"""

using DelimitedFiles

function check_psd_constraints(multipliers_array::Vector)
    counter = 0
    M = []
    for mults in multipliers_array
        for lms in mults
            for lm in lms
                if !isposdef(Matrix(lm.Q)) && !isposdef(Diagonal(svd(Matrix(lm.Q)).S))
                    counter += 1
                else 
#                     display(Matrix(lm.Q))
                    push!(M, [Matrix(lm.Q)])
                end
            end
        end
    end
    open("matrices_p4.txt", "w") do io
        writedlm(io, M)
    end
    if counter==0
        println("All constraints are PSD: Barrier certified")
    else
        println("PSD check failed!")
    end
end

In [ ]:
all_barrier_plus = push!([first_level_barriers], second_level_barriers);

In [ ]:
lms=[]
for i in all_barrier_plus
    if (!ismissing(i[1][4]))
        lms = append!(lms,i[1][4])
    end
end

In [ ]:
check_psd_constraints([lm0,lm1,lms])